In [14]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

import warnings
warnings.filterwarnings('ignore')

import pickle
import matplotlib.pyplot as plt
%matplotlib inline
#show version that is being operated
import qiskit
qiskit.__qiskit_version__
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-04-11 00:13:24,548: Credentials are already in use. The existing account in the session will be replaced.


In [21]:
from qiskit import IBMQ, assemble
from qiskit.tools.jupyter import backend_overview, backend_monitor
from qiskit.tools.monitor import job_monitor

#pulse_lib is where the waveform will be defined Gaussian, square, arbitrary shape
import qiskit.pulse as pulse
from qiskit.pulse import library as pulse_lib, Schedule, \
                        DriveChannel, MeasureChannel, AcquireChannel, MemorySlot, \
                        Acquire

In [22]:
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-04-11 00:36:06,092: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [26]:
#provider = IBMQ.get_provider(hub='ibm-q-internal', group='support',
#project='q-start-team')
#backend = provider.get_backend('ibmq_armonk')#or ibmq_alamden
#backend_config = backend.configuration()

from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')#or almalden 

ibmqfactory.load_account:WARNING:2021-04-11 00:50:38,497: Credentials are already in use. The existing account in the session will be replaced.


In [28]:
#verify that the backend supports Pulse features
#by checking the backend configuration
backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

dt = backend_config.dt
print(f"Sampling time: {dt*1e9} ns") #config ret dt in seconds multiply by
                                        # 1e9 to get nanoseconds
    #sampling t is the resolution of the backend
    #using armonk v almaden require adjust of measurement time

Sampling time: 0.2222222222222222 ns


In [29]:
backend
#shows what backend looks like below

<IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>

In [31]:
backend_defaults = backend.defaults()

In [32]:
Qubit_Frequency = backend_defaults.qubit_freq_est[0]
print ("The Qubit frequency is %.3f GHZ"%(Qubit_Frequency*(10**-9)))

The Qubit frequency is 4.972 GHZ


In [33]:
#Calculate amplitude form complex signal
#measurement level 1, this is a way to take those signals and extract
#the amplitudes of the inq(?) quadratures
def get_ampliptude(vec):
    i_signal = np.imag(vec)
    r_signal = np.real(vec)
    
    mvec = [np.mean(r_signal), np.mean(i_signal)]
    
    src_mat = np.vstack((r_signal - mvec [0], i_signal - mvec[1])).T
    (_, _, v_mat) = np.linalg.svd(src_mat)
    
    dvec = v_mat[0, 0:2]
    
    if dvec.dot(mvec) < 0:
        dvec = -dvec
        
    return src_mat.dot(dvec)

In [34]:
#use default backend gates to calibrate descriminator
calibrations = backend.defaults().instruction_schedule_map

In [35]:
qubit = 0
#x_gate pulses qubit from 0 to 1 state
#gate defined by u3 operation and appropriate angles
x_gate = calibrations.get('u3', [qubit], P0=np.pi, P1=0.0, P2=np.pi)
measure_gate = calibrations.get('measure', 
                                qubits=backend_config.meas_map[qubit])

In [36]:
drive_chan = DriveChannel(qubit)
meas_chan = MeasureChannel(qubit)
acq_chan = AcquireChannel(qubit)


In [37]:
#this is in dt, which is approx 150 ns
measure_time = 50

In [49]:
#create schedule
schedule = Schedule(name='Distribution Comparisons')
schedule += x_gate
schedule |= measure_gate << measure_time

In [51]:
#draw does not work, update, import or ?
schedule.draw(plot_range=[0,75], channels_to_plot=[drive_chan, meas_chan, acq_chan], label=True, scaling=10.0)

Traceback (most recent call last):
  File "<ipython-input-51-2e0f5a466577>", line 1, in <module>
    schedule.draw(plot_range=[0,75], channels_to_plot=[drive_chan, meas_chan, acq_chan], label=True, scaling=10.0)
  File "/opt/conda/lib/python3.8/site-packages/qiskit/pulse/schedule.py", line 1422, in wrapper
    return method(*args, **kwargs)
TypeError: draw() got an unexpected keyword argument 'channels_to_plot'

Use %tb to get the full traceback.
